In [1]:
!pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn

In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import numpy as np
import os
import evaluate

2025-05-30 07:44:12.089109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748591052.546801      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748591052.663808      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import load_dataset

dataset = load_dataset("taidng/UIT-ViQuAD2.0")
dataset

README.md:   0%|          | 0.00/6.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.20M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/735k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28454 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3814 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7301 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 28454
    })
    validation: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 3814
    })
    test: Dataset({
        features: ['id', 'uit_id', 'title', 'context', 'question', 'answers', 'is_impossible', 'plausible_answers'],
        num_rows: 7301
    })
})

In [4]:
# Tổng số phần tử trong tập train gốc
total_len = len(dataset["train"])
test_size = total_len // 10  # Lấy 1/10

# Chia tập train: 9/10 đầu giữ làm train, 1/10 cuối làm test
train_dataset = dataset["train"].select(range(0, total_len - test_size))
test_dataset = dataset["train"].select(range(total_len - test_size, total_len))
val_dataset = dataset["validation"]

# Kiểm tra kết quả
print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")
print(f"Test size: {len(test_dataset)}")


Train size: 25609
Validation size: 3814
Test size: 2845


In [5]:

print("Context: ", train_dataset[0]["context"])
print("Question: ", train_dataset[0]["question"])
print("Answer: ", train_dataset[0]["answers"])

Context:  Phạm Văn Đồng (1 tháng 3 năm 1906 – 29 tháng 4 năm 2000) là Thủ tướng đầu tiên của nước Cộng hòa Xã hội chủ nghĩa Việt Nam từ năm 1976 (từ năm 1981 gọi là Chủ tịch Hội đồng Bộ trưởng) cho đến khi nghỉ hưu năm 1987. Trước đó ông từng giữ chức vụ Thủ tướng Chính phủ Việt Nam Dân chủ Cộng hòa từ năm 1955 đến năm 1976. Ông là vị Thủ tướng Việt Nam tại vị lâu nhất (1955–1987). Ông là học trò, cộng sự của Chủ tịch Hồ Chí Minh. Ông có tên gọi thân mật là Tô, đây từng là bí danh của ông. Ông còn có tên gọi là Lâm Bá Kiệt khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm (Chủ nhiệm là Hồ Học Lãm).
Question:  Tên gọi nào được Phạm Văn Đồng sử dụng khi làm Phó chủ nhiệm cơ quan Biện sự xứ tại Quế Lâm?
Answer:  {'text': ['Lâm Bá Kiệt'], 'answer_start': [507]}


In [ ]:
from huggingface_hub import login

login(os.getenv("HF_TOKEN"))  # Dán token bạn vừa copy vào đây


In [7]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AddedToken

model_name = "google-bert/bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer.mask_token = AddedToken("<mask>", lstrip=True, normalized=True)
tokenizer.save_pretrained("Finetune_mBERT_QA")

model = AutoModelForQuestionAnswering.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def prepare_features(examples):
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized["offset_mapping"]
    example_ids = []
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized["input_ids"][i]
        sample_idx = sample_mapping[i]
        answer = examples["answers"][sample_idx]
        is_impossible = examples["is_impossible"][sample_idx]

        example_ids.append(examples["id"][sample_idx])

        if is_impossible or len(answer["text"]) == 0:
            start_positions.append(0)
            end_positions.append(0)
            tokenized["offset_mapping"][i] = [(0, 0)] * len(offsets)  # Đánh dấu là không hợp lệ
        else:
            start_char = answer["answer_start"][0]
            end_char = start_char + len(answer["text"][0])
            sequence_ids = tokenized.sequence_ids(i)

            context_start = sequence_ids.index(1)
            context_end = len(sequence_ids) - 1 - list(reversed(sequence_ids)).index(1)

            if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
                tokenized["offset_mapping"][i] = [(0, 0)] * len(offsets)
            else:
                idx = context_start
                while idx <= context_end and offsets[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offsets[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    tokenized["example_id"] = example_ids
    return tokenized


In [9]:
# Lấy 3 ví dụ đầu tiên
subset = train_dataset.select(range(100,120))
tokenized = prepare_features(subset)

for i in range(3):  # In 3 ví dụ đầu
    print(f"\nExample {i}")
    input_ids = tokenized["input_ids"][i]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    text = tokenizer.convert_tokens_to_string(tokens)
    
    print("Tokens:", tokens)
    print("Reconstructed text:", text)
    print("Start position:", tokenized["start_positions"][i])
    print("End position:", tokenized["end_positions"][i])



Example 0
Tokens: ['[CLS]', 'Tần', '##g', 'phát', 'sinh', 'b', '##ó', 'mạch', 'là', 'gì', '?', '[SEP]', 'Trong', 'nhóm', 'lớn', 'hơn', 'là', 'thực', 'vật', 'hai', 'lá', 'm', '##ầm', 'thì', 'chúng', 'có', 'các', 'b', '##ó', 'mạch', 'trong', 'các', 'thân', 'cây', 'non', 'được', 's', '##ắp', 'xếp', 'trong', 'vòng', 'gỗ', 'mở', ',', 'chia', 'tách', 'phần', 'lõi', 'x', '##ố', '##p', 'trung', 'tâm', 'với', 'phần', 'v', '##ỏ', 'ngoài', '.', 'Trong', 'mỗi', 'b', '##ó', ',', 'chia', 'tách', 'bởi', 'phần', 'chất', 'gỗ', 'và', 'phần', 'li', 'be', ',', 'là', 'một', 'lớp', 'mô', 'phân', 'sinh', 'hay', 'mô', 'hình', 'thành', 'đang', 'hoạt', 'động', ',', 'được', 'biết', 'dưới', 'tên', 'gọi', 'tầng', 'phát', 'sinh', 'gỗ', ';', 'bằng', 'sự', 'hình', 'thành', 'của', 'lớp', 'phát', 'sinh', 'gỗ', 'giữa', 'các', 'b', '##ó', '(', 'tầng', 'phát', 'sinh', 'gỗ', 'trong', 'b', '##ó', ')', 'thì', 'vòng', 'gỗ', 'hoàn', 'h', '##ảo', 'được', 'tạo', 'ra', ',', 'và', 'đều', 'đ', '##ặ', '##n', 'tăng', 'độ', 'dà', '##

In [10]:
tokenized_train = train_dataset.map(prepare_features, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(prepare_features, batched=True, remove_columns=val_dataset.column_names)


Map:   0%|          | 0/25609 [00:00<?, ? examples/s]

Map:   0%|          | 0/3814 [00:00<?, ? examples/s]

In [11]:
# Load raw validation
raw_validation_dataset = dataset["validation"]
tokenized_validation = tokenized_val

# Load metric
squad_metric = evaluate.load("squad_v2")

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size=20, max_answer_length=30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k["id"]: i for i, k in enumerate(examples)}
    features_per_example = collections.defaultdict(list)

    for i, feature in enumerate(features):
        features_per_example[feature["example_id"]].append(i)

    predictions = collections.OrderedDict()

    for example in examples:
        example_id = example["id"]
        context = example["context"]
        feature_indices = features_per_example[example_id]

        best_score = -float("inf")
        best_answer = ""

        for i in feature_indices:
            start_logits = all_start_logits[i]
            end_logits = all_end_logits[i]
            offset_mapping = features[i]["offset_mapping"]

            for start_index in np.argsort(start_logits)[-n_best_size:]:
                for end_index in np.argsort(end_logits)[-n_best_size:]:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    score = start_logits[start_index] + end_logits[end_index]

                    if score > best_score:
                        best_score = score
                        best_answer = context[start_char:end_char]

        if example.get("is_impossible", False) or best_answer.strip() == "":
            predictions[example_id] = ""
        else:
            predictions[example_id] = best_answer.strip()

    return predictions




In [12]:
def compute_metrics(p):
    predictions = postprocess_qa_predictions(
        dataset["validation"],
        tokenized_val,
        p.predictions
    )

    formatted_predictions = [
        {
            "id": k,
            "prediction_text": v,
            "no_answer_probability": 0.0
        }
        for k, v in predictions.items()
    ]

    references = [
        {
            "id": ex["id"],
            "answers": ex["answers"]
            if ex["answers"]["text"]
            else {"text": [""], "answer_start": [0]}
        }
        for ex in dataset["validation"]
    ]

    return squad_metric.compute(predictions=formatted_predictions, references=references)


In [13]:
print(tokenized_train)


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_id'],
    num_rows: 27392
})


In [14]:
print(model)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding
import os
import collections
from transformers import EarlyStoppingCallback
os.environ["WANDB_DISABLED"] = "true"

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
#model_name = "CS338-" + model_id.split("/")[-1]+"-finetuned-xlm-roberta-large"
# Updated Training Arguments
training_args = TrainingArguments(
    output_dir="/kaggle/working/Finetune_mBERT_QA",
    
    # Epoch-based saving and evaluation
    eval_strategy="epoch",   # Changed from steps to epoch
    save_strategy="epoch",         # Save checkpoint at end of each epoch
    save_total_limit=2,            # Keep maximum 2 checkpoints
    
    # Multi-GPU Configuration
    ddp_find_unused_parameters=False,  # Important for multi-GPU training
    dataloader_num_workers=2,          # Dataloader workers
    
    # Training Hyperparameters
    learning_rate=2e-5,
    num_train_epochs=10,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    
    # Optimization Parameters
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=2,
    
    # Model Selection and Logging
    load_best_model_at_end=True,   # Automatically load best model
    logging_steps=50,
    logging_dir="/kaggle/working/logs",
    metric_for_best_model="f1",    # Select best model based on F1 score
    greater_is_better=True,        # Higher F1 is better
    
    # Ensure multi-GPU training
    dataloader_pin_memory=True,
    #push_to_hub=True,
    # Push to Hugging Face Hub
    push_to_hub=True,              # Enable pushing the model to Hugging Face Hub
    hub_strategy="end",            # When to push: 'end' means push after training is done
    hub_model_id="DatTran0509/Finetune_mBERT_QA",  # Specify your Hugging Face username and model name
    hub_token=os.getenv("HF_TOKEN")  # Add your Hugging Face API token here
)

# Trainer Configuration
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # Add EarlyStoppingCallback with patience
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Set patience to 2
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipykernel_19/2487428320.py:51: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
trainer.train()



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Exact,F1,Total,Hasans Exact,Hasans F1,Hasans Total,Best Exact,Best Exact Thresh,Best F1,Best F1 Thresh
1,1.715300,1.691615,61.719979,71.688305,3814,61.719979,71.688305,3814,61.719979,0.000000,71.688305,0.000000
2,1.399500,1.555866,57.262716,64.300376,3814,57.262716,64.300376,3814,57.262716,0.000000,64.300376,0.000000
3,1.214700,1.470418,64.866282,74.818655,3814,64.866282,74.818655,3814,64.866282,0.000000,74.818655,0.000000
4,1.046500,1.536757,65.180912,75.729193,3814,65.180912,75.729193,3814,65.180912,0.000000,75.729193,0.000000
5,0.929100,1.552880,65.154693,75.100870,3814,65.154693,75.100870,3814,65.154693,0.000000,75.100870,0.000000
6,0.810700,1.663322,65.495543,76.567045,3814,65.495543,76.567045,3814,65.495543,0.000000,76.567045,0.000000
7,0.721100,1.664690,64.525433,74.899138,3814,64.525433,74.899138,3814,64.525433,0.000000,74.899138,0.000000
8,0.668000,1.773952,64.604090,74.929624,3814,64.604090,74.929624,3814,64.604090,0.000000,74.929624,0.000000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=1712, training_loss=1.135862730373846, metrics={'train_runtime': 9137.1529, 'train_samples_per_second': 29.979, 'train_steps_per_second': 0.234, 'total_flos': 4.294464456307507e+16, 'train_loss': 1.135862730373846, 'epoch': 8.0})

In [17]:
trainer.save_model("/kaggle/working/Finetune_mBERT_QA")
tokenizer.save_pretrained("/kaggle/working/Finetune_mBERT_QA")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/709M [00:00<?, ?B/s]

('/kaggle/working/Finetune_mBERT_QA/tokenizer_config.json',
 '/kaggle/working/Finetune_mBERT_QA/special_tokens_map.json',
 '/kaggle/working/Finetune_mBERT_QA/vocab.txt',
 '/kaggle/working/Finetune_mBERT_QA/added_tokens.json',
 '/kaggle/working/Finetune_mBERT_QA/tokenizer.json')

In [18]:
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer
# import torch

# # Khởi tạo model và tokenizer
# model_id = "DatTran0509/Finetune_XLM_R_base_QA"
# model = AutoModelForQuestionAnswering.from_pretrained(model_id)
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# # Tùy chỉnh context và câu hỏi
# context = """
# Ông Nguyễn Ngọc Hà cũng nêu một số điểm mới trong công tác tổ chức thi. Theo đó, một trong những vấn đề mà các sở lo lắng là quá trình sắp xếp phòng thi. Kỳ thi chỉ tổ chức 3 buổi thi, đặc biệt buổi thứ 3 chỉ có 1 buổi nhưng có 2 môn thi. Ví dụ môn vật lý có thể thi trong 2 khung giờ trong buổi thi đó. Việc này được xử lý ra sao?
# Để tiện lợi cho học sinh, các em được sắp xếp ở một phòng thi duy nhất, không phải di chuyển trong các buổi thi. Cho dù một môn thi ở hai khung giờ khác nhau, để đảm bảo yếu tố an ninh thay vì chỉ có 24 mã đề thì nâng lên thành 48 mã đề, 24 mã/khung giờ.
# """
# question = "Thi bao nhiêu ngày?"

# # Tokenize
# inputs = tokenizer(
#     question,
#     context,
#     return_tensors="pt",
#     truncation=True,
#     max_length=384,
#     return_offsets_mapping=True
# )

# offset_mapping = inputs.pop("offset_mapping")  # Bỏ để không lỗi khi đưa vào model

# # Dự đoán với model
# with torch.no_grad():
#     outputs = model(**inputs)
#     start_logits = outputs.start_logits
#     end_logits = outputs.end_logits

# # Tìm vị trí có xác suất cao nhất
# start_index = torch.argmax(start_logits)
# end_index = torch.argmax(end_logits) + 1

# # Kiểm tra nếu end_index nhỏ hơn hoặc bằng start_index, tức là không tìm thấy câu trả lời hợp lệ
# if end_index <= start_index:
#     answer = ""
# else:
#     offsets = offset_mapping[0]
#     start_char = offsets[start_index][0].item()
#     end_char = offsets[end_index - 1][1].item()
#     answer = context[start_char:end_char]

# # In kết quả
# print(f"❓ Câu hỏi: {question}")
# print(f"📄 Context: {context[:300]}...")  # Cắt context để dễ đọc (300 ký tự đầu)

# # Kiểm tra nếu không có câu trả lời
# if not answer:
#     print(f"✅ Trả lời: [KHÔNG CÓ CÂU TRẢ LỜI TRONG ĐOẠN VĂN]")
# else:
#     print(f"✅ Trả lời: {answer}")
